### Importando bibliotecas 

In [1]:
import cvxpy as cp
import numpy as np
import mosek



### Definindo problema de otimização: variáveis e constraints

#### Variáveis

In [71]:
def getCoeffs(beta=0.5):
    _linGamma = (1/2 - beta + (beta-1)/4)
    _indGamma = beta + (1-beta)/4
    _linDelta = (1/2 + (beta-1)/4)
    _linEpsilon = (beta-1)/4
    _indDelta = _indEpsilon = (1-beta)/4

    linearCoeffs = [_linGamma, _linGamma, _linGamma, _linGamma-1/2,
                _linEpsilon, _linEpsilon, _linEpsilon, _linDelta,
                _linEpsilon, _linEpsilon, _linEpsilon, _linDelta,
                _linDelta, _linDelta, _linDelta, _linEpsilon]

    independentCoeffs = [_indGamma, _indGamma, _indGamma, _indGamma,
                _indEpsilon, _indEpsilon, _indEpsilon, _indDelta,
                _indEpsilon, _indEpsilon, _indEpsilon, _indDelta,
                _indDelta, _indDelta, _indDelta, _indEpsilon]
    
    return linearCoeffs, independentCoeffs

betas = np.linspace(start=0, stop=1, num=20)
alphas = []

beta = 0.2

constraints = []

p = cp.Variable(shape=16)
constraints += [p >= 0, p <= 1]

alpha = cp.Variable(shape=1)
constraints += [alpha >= 0, alpha <= 1]


# Computes values for the coefficients on the p(ab|xy) distribution given in the exercise
linearCoeffs, independentCoeffs = getCoeffs(beta)


# Fine's theorem constraints for locality
constraints += [alpha*linearCoeffs[0:4]+ independentCoeffs[0:4] == cp.sum(p[0:4])*np.ones(4),
                alpha*linearCoeffs[4:8] + independentCoeffs[4:8] == cp.sum(p[4:8])*np.ones(4),
                alpha*linearCoeffs[8:12] + independentCoeffs[8:12] == cp.sum(p[8:12])*np.ones(4),
                alpha*linearCoeffs[12:16] + independentCoeffs[12:16] == cp.sum(p[12:16])*np.ones(4)]


# Probability distribution sums to one
constraints += [cp.sum(p) == 1]

objective = cp.Minimize(alpha)
problem = cp.Problem(objective=objective, constraints=constraints)
print(problem)

print("Starting optimization")
resultado = problem.solve(solver="MOSEK", verbose=True)
if(resultado != None):
    optimalPoint = alpha.value
    optimumDistribution = p.value
    print("Optimum of found at alpha = {}".format(optimalPoint))
    print("Joint distribution values: {}".format(optimumDistribution))
    alphas += [alpha.value[0]]
else:
    print("Infeasible or unbounded problem.")  
    alphas += [None] 



minimize var51639
subject to 0.0 <= var51628
           var51628 <= 1.0
           0.0 <= var51639
           var51639 <= 1.0
           Promote(var51639, (4,)) @ [ 0.1  0.1  0.1 -0.4] + [0.4 0.4 0.4 0.4] == Promote(Sum(var51628[0:4], None, False), (4,)) @ [1. 1. 1. 1.]
           Promote(var51639, (4,)) @ [-0.2 -0.2 -0.2  0.3] + [0.2 0.2 0.2 0.2] == Promote(Sum(var51628[4:8], None, False), (4,)) @ [1. 1. 1. 1.]
           Promote(var51639, (4,)) @ [-0.2 -0.2 -0.2  0.3] + [0.2 0.2 0.2 0.2] == Promote(Sum(var51628[8:12], None, False), (4,)) @ [1. 1. 1. 1.]
           Promote(var51639, (4,)) @ [ 0.3  0.3  0.3 -0.2] + [0.2 0.2 0.2 0.2] == Promote(Sum(var51628[12:16], None, False), (4,)) @ [1. 1. 1. 1.]
           Sum(var51628, None, False) == 1.0
Starting optimization
                                     CVXPY                                     
                                     v1.5.1                                    
(CVXPY) Jun 10 09:27:44 PM: Your problem has 17 variables, 51 co

#### var

In [68]:
def getCoeffs(beta=0.5):
    _linGamma = (1/2 - beta + (beta-1)/4)
    _indGamma = beta + (1-beta)/4
    _linDelta = (1/2 + (beta-1)/4)
    _linEpsilon = (beta-1)/4
    _indDelta = _indEpsilon = (1-beta)/4

    linearCoeffs = [_linGamma, _linGamma, _linGamma, _linGamma-1/2,
                _linEpsilon, _linEpsilon, _linEpsilon, _linDelta,
                _linEpsilon, _linEpsilon, _linEpsilon, _linDelta,
                _linDelta, _linDelta, _linDelta, _linEpsilon]

    independentCoeffs = [_indGamma, _indGamma, _indGamma, _indGamma,
                _indEpsilon, _indEpsilon, _indEpsilon, _indDelta,
                _indEpsilon, _indEpsilon, _indEpsilon, _indDelta,
                _indDelta, _indDelta, _indDelta, _indEpsilon]
    
    return linearCoeffs, independentCoeffs

betas = np.linspace(start=0, stop=1, num=20)
alphas = []

# beta = 0.5
for beta in betas:
    constraints = []

    p = cp.Variable(shape=16)
    constraints += [p >= 0, p <= 1]

    alpha = cp.Variable(shape=1)
    constraints += [alpha >= 0, alpha <= 1]


    # Computes values for the coefficients on the p(ab|xy) distribution given in the exercise
    linearCoeffs, independentCoeffs = getCoeffs(beta)


    # Fine's theorem constraints for locality
    constraints += [alpha*linearCoeffs[0:4]+ independentCoeffs[0:4] == cp.sum(p[0:4])*np.ones(4),
                    alpha*linearCoeffs[4:8] + independentCoeffs[4:8] == cp.sum(p[4:8])*np.ones(4),
                    alpha*linearCoeffs[8:12] + independentCoeffs[8:12] == cp.sum(p[8:12])*np.ones(4),
                    alpha*linearCoeffs[12:16] + independentCoeffs[12:16] == cp.sum(p[12:16])*np.ones(4)]


    # Probability distribution sums to one
    constraints += [cp.sum(p) == 1]

    objective = cp.Maximize(alpha)
    problem = cp.Problem(objective=objective, constraints=constraints)
    print(problem)

    print("Starting optimization")
    resultado = problem.solve(solver="MOSEK", verbose=False)
    if(resultado != None):
        optimalPoint = alpha.value
        optimumDistribution = p.value
        print("Optimum of found at alpha = {}".format(optimalPoint))
        print("Joint distribution values: {}".format(optimumDistribution))
        alphas += [alpha.value[0]]
    else:
        print("Infeasible or unbounded problem.")  
        alphas += [None] 

print(alphas)

maximize var48147
subject to 0.0 <= var48136
           var48136 <= 1.0
           0.0 <= var48147
           var48147 <= 1.0
           Promote(var48147, (4,)) @ [ 0.25  0.25  0.25 -0.25] + [0.25 0.25 0.25 0.25] == Promote(Sum(var48136[0:4], None, False), (4,)) @ [1. 1. 1. 1.]
           Promote(var48147, (4,)) @ [-0.25 -0.25 -0.25  0.25] + [0.25 0.25 0.25 0.25] == Promote(Sum(var48136[4:8], None, False), (4,)) @ [1. 1. 1. 1.]
           Promote(var48147, (4,)) @ [-0.25 -0.25 -0.25  0.25] + [0.25 0.25 0.25 0.25] == Promote(Sum(var48136[8:12], None, False), (4,)) @ [1. 1. 1. 1.]
           Promote(var48147, (4,)) @ [ 0.25  0.25  0.25 -0.25] + [0.25 0.25 0.25 0.25] == Promote(Sum(var48136[12:16], None, False), (4,)) @ [1. 1. 1. 1.]
           Sum(var48136, None, False) == 1.0
Starting optimization
Optimum of found at alpha = [-0.]
Joint distribution values: [0.05760227 0.06413258 0.06413258 0.06413258 0.05760227 0.06413258
 0.06413258 0.06413258 0.05760227 0.06413258 0.06413258 0.064132

<generator object <genexpr> at 0x7ae10cb52a40>


## Programação Semi-Definida:

In [8]:
# No caso da programção semi-definida, seguimos os mesmo passos do exercício anterior
# Aqui no entanto, nossas variáveis serão componentes de uma matriz simétrica

# código feio horroroso para construir as matrizes do problema
A = np.zeros((4,4))
B = np.zeros((4,4))
Id = np.identity(4)

A[0,3] = A[1,2] = A[2,3] = A[3,0] = B[0,0] = B[3,3] = 1
B[1,1] = B[2,2] = -1

# instanciando uma matriz simétrica como variável de decisão
X = cp.Variable((4,4), symmetric=True)

# criando lista de constraints para o cvxpy
constraints = [cp.trace(B @ X) == 1, cp.trace(X) == 1, X >> 0]

# instanciando e resolvendo problema 
objective = cp.Maximize(cp.trace(A @ X))
prob = cp.Problem(objective=objective, constraints=constraints)
prob.solve(solver="MOSEK")

# mostrando resultados
# truncamos o resultado final pois haviam pequenos devios nos valores, muito provavelmente advindos de erros de ponto flutuante
print("The optimal value is", '%.3f'%(prob.value))
print("A solution X is")
print(X.value.round(3))

The optimal value is 1.000
A solution X is
[[ 0.5  0.   0.   0.5]
 [ 0.  -0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.5  0.   0.   0.5]]
